<a href="https://colab.research.google.com/github/TurkuNLP/textual-data-analysis-course/blob/main/ir_and_boolean_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information Retrieval

* Text search engines
  * Term-document matrices / inverted index
  * Dense vector representations, especially NN-based


**Disclaimer:** some of the material is borrowed with thanks from http://www.cis.lmu.de/~hs/teach/14s/ir/

# Boolean model

* The Boolean model is arguably the simplest model to
  base an information retrieval system on
* Still relevant b
* Queries are Boolean expressions, e.g., *Caesar* and *Brutus*
* The seach engine returns all documents that satisfy the Boolean expression, but there are exceptions:
  * e.g. page contains variant of query words: morphology, spelling correction, synonyms

## Term-document matrix

* Rows: terms
* Columns: document
* $M_{ij}=1$ if term *i* present in document *j*


In [1]:
# We will not be using much of sklearn in the course, but this is an easy way to illustrate
# the basic ideas
# soon enough we will switch to the transformers code and stay there

from sklearn.feature_extraction.text import CountVectorizer
documents=["This is a silly example","A better example","Nothing to see here","This is a great and long example"]
cv=CountVectorizer(lowercase=False,binary=True)
print("Term-document matrix:\n")
td_matrix=cv.fit_transform(documents).todense().T   #.T transposes the matrix, sklearn maintains document-term
print(td_matrix)
print("\nIDX -> terms mapping:\n")
print(cv.get_feature_names())
print("\nterm -> IDX mapping:\n")
print(cv.vocabulary_) # note the _ at the end


Term-document matrix:

[[0 0 1 0]
 [1 0 0 1]
 [0 0 0 1]
 [0 1 0 0]
 [1 1 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [1 0 0 1]
 [0 0 0 1]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 1 0]]

IDX -> terms mapping:

['Nothing', 'This', 'and', 'better', 'example', 'great', 'here', 'is', 'long', 'see', 'silly', 'to']

term -> IDX mapping:

{'This': 1, 'is': 7, 'silly': 10, 'example': 4, 'better': 3, 'Nothing': 0, 'to': 11, 'see': 9, 'here': 6, 'great': 5, 'and': 2, 'long': 8}


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


* Every row is an *incidence vector* of a term - which documents the term appears in
* Inverted index -> direct path from a term to documents
* Boolean retrieval in its simplest form:
  * Boolean operations on incidence vectors

In [2]:
t2i=cv.vocabulary_
print("example")
print(td_matrix[t2i["example"]])
print()
print("example and great")
print(td_matrix[t2i["example"]] & td_matrix[t2i["great"]])
print()
print("not example")
print(1-td_matrix[t2i["example"]]) #1-x does the negation in our case
print()

example
[[1 1 0 1]]

example and great
[[0 0 0 1]]

not example
[[0 0 1 0]]



## DIY search engine

We can piece it together into an elementary search engine, you can check it out, but we won't go through this in the course

* Accept queries like "( not example or great ) and Nothing"
* Rewrite them into a Python expression
* eval() that expression

In [3]:
# Operators and, or, not become &, |, 1 -
# Parentheses are left untouched
# Everything else interpreted as a term and fed through td_matrix[t2i["..."]]

d={"and":"&","or":"|","not":"1 -","(":"(",")":")"} # operator replacements
def rew_token(t):
    return d.get(t,f'td_matrix[t2i["{t}"]]') #if it is operator, replace, if not, then replace with td_matrix[t2i["{t}"]]

def rew_query(query): #rewrite every token in the query
    return " ".join(rew_token(t) for t in query.split())

def test_query(query):
    print("Query:\""+query+"\"")
    print("Rewritten:",rew_query(query))
    print("Matching:",eval(rew_query(query)))
    print()

test_query("example and not Nothing")
test_query("not example or great")
test_query("( not example or great ) and Nothing")


Query:"example and not Nothing"
Rewritten: td_matrix[t2i["example"]] & 1 - td_matrix[t2i["Nothing"]]
Matching: [[1 1 0 1]]

Query:"not example or great"
Rewritten: 1 - td_matrix[t2i["example"]] | td_matrix[t2i["great"]]
Matching: [[0 0 1 1]]

Query:"( not example or great ) and Nothing"
Rewritten: ( 1 - td_matrix[t2i["example"]] | td_matrix[t2i["great"]] ) & td_matrix[t2i["Nothing"]]
Matching: [[0 0 1 0]]



In [ ]:
# Should match all documents, but crashes instead
# This one you will fix as an exercise
import traceback
try:
    test_query("not awordwhichdoesnotexist") #should match all documents
except:
    traceback.print_exc() #I only need to do this because of IPython

Query:"not awordwhichdoesnotexist"
Rewritten: 1 - td_matrix[t2i["awordwhichdoesnotexist"]]


Traceback (most recent call last):
  File "<ipython-input-25-a02963e0a344>", line 5, in <module>
    test_query("not awordwhichdoesnotexist") #should match all documents
  File "<ipython-input-24-ade8113ddedf>", line 15, in test_query
    print("Matching:",eval(rew_query(query)))
  File "<string>", line 1, in <module>
KeyError: 'awordwhichdoesnotexist'


* We now have a rudimentary boolean IR system
* It is not that great but ready in < 10 lines of code in Python

# Limitations of this elementary implementation (of this elementary model)

## Size constraints
* Term-document matrix is vocabulary size times document set size
* 1M documents, each 1000 words
* 1,000,000 x 1,000 = 1,000,000,000 words of running text
* 6 bytes per word -> ~6GB in size
* Assume 500,000 unique terms
* Term-document matrix has 1,000,000 x 500,000 / 8 / 1024 / 1024 / 1024 -> ~60GB
* 60GB of space to index a collection of 6GB of text!
* ...but most of this are zeros...
* Sparse representation: only remember the non-zero entries
* For every term, remember a (usually sorted) list of documents in which it appears
* This is the famous **inverted index**
* Scipy sparse formats: https://docs.scipy.org/doc/scipy/reference/sparse.html
  * *CSC* for every column remember the list of rows
  * *CSR* for every row remember the list of columns
  


## Phrase and proximity queries

* "*Stanford university*" - not the same thing as *Stanford AND university*
* The basic inverted index of no help
* Index all word bigrams
  * "*Stanford university*" becomes a term
  * but then you have problem with *"Stanford university courses"*
  * "to be or not to be" -> "to be" and "be or" and "or not" and "not to" + postprocessing
  * Not a great solution:
  * Massive waste of space: blows up the index size quadratically
  * Needs postprocessing to weed out the false hits
* Positional index
  * For every term, and every document, remember a list of the positions, not just "1"
  * A modification of the sorted list intersection algorithm to answer the query
  * Also allows proximity queries "X within N words from Y"
  * Quite heavy computationally
* Combined index
  * Most common biwords indexed directly
  * Rest solved with positional indexing
  * Compromise between the two

# Result ranking

* Obviously useful for large document collections
* Come up with a number describing the fit of a document to a query
* Return top-N documents
* Basic observations:
  * The more query terms hit, the more relevant the document is
  * The more times the query terms hit in the document, the more relevant the document is
  * Rare terms are more informative than common terms
* Do not store 0/1, store count / TF-IDF weights

### Informativeness of terms

* Not all words are equally informative
* Rare words are clearly much more informative than common ones
* The query *"jabberwocky movie cast"* should put more weight on *jabberwocky*
* What we want:
  * High positive weight for rare terms
  * Low positive weight for common terms
* Typical way: IDF *inverse document frequency*
  * df_t is the *document frequency* of *t* - number of documents *t* appear in
  * $IDF_t=\frac{N}{df_t}$ inverse document frequency of *t*
  * Usually one would squeeze this through log so
  * $IDF_t=log_{10}(\frac{N}{df_t})$

## Tf.Idf

* An extremely common weighting scheme in IR
* Product of term's tf (log-squeezed) with the term's idf (log-squeezed)
* Gives a score of that term's hit in a given document
  * $(1+log_{10}tf_t)\cdot log_{10}\frac{N}{df_t}$
  

# Vector-space model

* So far we looked at term vectors (rows of the term-document matrix)
* We could also think about document vectors (columns of the term-document matrix)
* Documents are vectors in a high-dimensional space, terms are the dimensions
* Document vectors very high-dimensional but very very sparse
* Same for queries - queries can also be seen as vectors in a high-dimensional space
* Search:
  * similarity between query vector and document vector
  * higher similarity means better hit (rank higher)
  
## Similarity measures

* Similarity -> negative distance
* Eucledian distance -> affected by vector length
* Cosine similarity -> cosine of the angle between query and document vectors
  * 1 for total similarity (angle 0)
  * -1 for complete opposite
  * we only have positive numbers in our vectors, so we are on the [0,1] scale not [-1,1]
  * not sensitive to length
  * incredibly easy to compute!
  * $cos(u,v)=\frac{u\cdot v}{||u||\cdot ||v||}=\frac{\sum_i v_i\cdot u_i}{\sqrt{\sum_i u_i^2}\sqrt{\sum_i v_i^2}}$
  * dot-product of normalized vectors - just multiply the numbers together, really
  
## Weighting schemes - document, query

* Now we can apply different weighting to documents and queries
* Typical choice:
  * document: log tf, no idf, length-normalized - why no idf?
  * query: log tf, log idf, length-normalized
  * cosine similarity

